<a href="https://colab.research.google.com/github/AnkitRajSri/FacialEmotionAI/blob/main/EmotionAIModelsDeployment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
import os
import pandas as pd
import numpy as np
import PIL
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
import pickle
from PIL import *
import cv2
import json
import tensorflow as tf
from tensorflow.keras import backend as K
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

In [4]:
%cd /content/drive/MyDrive/Emotion AI

/content/drive/MyDrive/Emotion AI


### Prediction Pipeline

##### Loading and compiling the models

In [7]:
def pretrained_model(struct_path, weight_path, is_expression=False):
  with open(struct_path, "r") as json_file:
      json_saved_model= json_file.read()
  model = models.model_from_json(json_saved_model)
  model.load_weights(weight_path)
  if is_expression:
    model_optimizer = "Adam"
    model_loss = "categorical_crossentropy"
  else:
    model_optimizer = optimizers.Adam(learning_rate=0.0001, beta_1=0.9, beta_2=0.999, amsgrad=False)
    model_loss = "mean_squared_error"
  model.compile(loss=model_loss, optimizer=model_optimizer , metrics = ["accuracy"])
  return model

In [10]:
facialkeypoints_model = pretrained_model("facialkeypoints_model.json", "best_facialkeypoints_weights.hdf5")
facialkeypoints_model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 96, 96, 1)]  0                                            
__________________________________________________________________________________________________
zero_padding2d (ZeroPadding2D)  (None, 102, 102, 1)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv_1 (Conv2D)                 (None, 48, 48, 64)   3200        zero_padding2d[0][0]             
__________________________________________________________________________________________________
bn_conv_1 (BatchNormalization)  (None, 48, 48, 64)   256         conv_1[0][0]                     
______________________________________________________________________________________________

In [11]:
facialexpression_model = pretrained_model("facialexpression_model.json", "best_facialexpression_weights.hdf5", is_expression=True)
facialexpression_model.summary()

Model: "facialexpression_resnet"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 96, 96, 1)]  0                                            
__________________________________________________________________________________________________
zero_padding2d (ZeroPadding2D)  (None, 102, 102, 1)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 48, 48, 64)   3200        zero_padding2d[0][0]             
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 48, 48, 64)   256         conv1[0][0]                      
____________________________________________________________________________

In [13]:
columns = ['left_eye_center_x', 'left_eye_center_y', 'right_eye_center_x',
       'right_eye_center_y', 'left_eye_inner_corner_x',
       'left_eye_inner_corner_y', 'left_eye_outer_corner_x',
       'left_eye_outer_corner_y', 'right_eye_inner_corner_x',
       'right_eye_inner_corner_y', 'right_eye_outer_corner_x',
       'right_eye_outer_corner_y', 'left_eyebrow_inner_end_x',
       'left_eyebrow_inner_end_y', 'left_eyebrow_outer_end_x',
       'left_eyebrow_outer_end_y', 'right_eyebrow_inner_end_x',
       'right_eyebrow_inner_end_y', 'right_eyebrow_outer_end_x',
       'right_eyebrow_outer_end_y', 'nose_tip_x', 'nose_tip_y',
       'mouth_left_corner_x', 'mouth_left_corner_y', 'mouth_right_corner_x',
       'mouth_right_corner_y', 'mouth_center_top_lip_x',
       'mouth_center_top_lip_y', 'mouth_center_bottom_lip_x',
       'mouth_center_bottom_lip_y']

##### Making predictions from the models

In [15]:
def predict(X_test):
  # Making predictions from the facialkeypoints model
  pred_keypoints = facialkeypoints_model.predict(X_test)

  # Making predictions from the expression model
  pred_expressions = np.argmax(facialexpression_model.predict(X_test), axis=-1)

  # Reshaping the array from (856, ) to (856, 1)
  pred_expressions = np.expand_dims(df_expression, axis=1)

  # Converting the predictions into a dataframe
  df_predict = pd.DataFrame(pred_keypoints, columns=columns)

  # Adding expression to the prediction dataframe
  df_predict["expression"] = pred_expressions

  return df_predict

### Model Deployment using Tensorflow Serving

##### Package the deployment steps in a function

In [17]:
def deploy(directory, model):
  MODEL_DIR = directory
  version = 1

  export_path = os.path.join(MODEL_DIR, str(version))
  print("Export path: {}\n".format(export_path))

  if os.path.isdir(export_path):
    print("\nAlready saved a model, cleaning up\n")
    !rm -r {export_path}

  tf.saved_model.save(model, export_path)
  os.environ["MODEL_DIR"] = MODEL_DIR

##### Add the tensorflow-model-server package to our list of packages

In [19]:
!echo "deb http://storage.googleapis.com/tensorflow-serving-apt stable tensorflow-model-server tensorflow-model-server-universal" | tee /etc/apt/sources.list.d/tensorflow-serving.list && \
curl https://storage.googleapis.com/tensorflow-serving-apt/tensorflow-serving.release.pub.gpg | apt-key add -
!apt update

deb http://storage.googleapis.com/tensorflow-serving-apt stable tensorflow-model-server tensorflow-model-server-universal
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2943  100  2943    0     0  81750      0 --:--:-- --:--:-- --:--:-- 81750
OK
Get:1 http://storage.googleapis.com/tensorflow-serving-apt stable InRelease [3,012 B]
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:4 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:8 http

##### Install tensorflow-model-server

In [20]:
!apt-get install tensorflow-model-server

Reading package lists... Done
Building dependency tree       
Reading state information... Done
Fetched 223 MB in 3s (72.9 MB/s)
Selecting previously unselected package tensorflow-model-server.
(Reading database ... 146442 files and directories currently installed.)
Preparing to unpack .../tensorflow-model-server_2.4.1_all.deb ...
Unpacking tensorflow-model-server (2.4.1) ...
Setting up tensorflow-model-server (2.4.1) ...


##### Deploy both the models

In [22]:
deploy("/model_1", facialkeypoints_model)

Export path: /model_1/1

INFO:tensorflow:Assets written to: /model_1/1/assets


In [24]:
%%bash --bg
nohup tensorflow_model_server \
  --rest_api_port=4500 \
  --model_name=facial_keypoints_model \
  --model_base_path="${MODEL_DIR}" > server.log 2>&1

Starting job # 0 in a separate thread.


In [25]:
!tail server.log

2021-02-08 21:40:13.159531: I external/org_tensorflow/tensorflow/cc/saved_model/loader.cc:206] Restoring SavedModel bundle.
2021-02-08 21:40:13.180537: I external/org_tensorflow/tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2299995000 Hz
2021-02-08 21:40:13.545565: I external/org_tensorflow/tensorflow/cc/saved_model/loader.cc:190] Running initialization op on SavedModel bundle at path: /model_1/1
2021-02-08 21:40:13.633294: I external/org_tensorflow/tensorflow/cc/saved_model/loader.cc:277] SavedModel load for tags { serve }; Status: success: OK. Took 666907 microseconds.
2021-02-08 21:40:13.649920: I tensorflow_serving/servables/tensorflow/saved_model_warmup_util.cc:59] No warmup data file found at /model_1/1/assets.extra/tf_serving_warmup_requests
2021-02-08 21:40:13.650416: I tensorflow_serving/core/loader_harness.cc:87] Successfully loaded servable version {name: facial_keypoints_model version: 1}
2021-02-08 21:40:13.654066: I tensorflow_serving/model_serve

In [27]:
deploy("/model_2", facialexpression_model)

Export path: /model_2/1


Already saved a model, cleaning up



In [28]:
%%bash --bg
nohup tensorflow_model_server \
  --rest_api_port=4000 \
  --model_name=facial_expressions_model \
  --model_base_path="${MODEL_DIR}" > server.log 2>&1

Starting job # 2 in a separate thread.


In [29]:
!tail server.log

2021-02-08 21:42:47.708940: I external/org_tensorflow/tensorflow/cc/saved_model/loader.cc:206] Restoring SavedModel bundle.
2021-02-08 21:42:47.722356: I external/org_tensorflow/tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2299995000 Hz
2021-02-08 21:42:48.117966: I external/org_tensorflow/tensorflow/cc/saved_model/loader.cc:190] Running initialization op on SavedModel bundle at path: /model_2/1
2021-02-08 21:42:48.245867: I external/org_tensorflow/tensorflow/cc/saved_model/loader.cc:277] SavedModel load for tags { serve }; Status: success: OK. Took 696736 microseconds.
2021-02-08 21:42:48.267269: I tensorflow_serving/servables/tensorflow/saved_model_warmup_util.cc:59] No warmup data file found at /model_2/1/assets.extra/tf_serving_warmup_requests
2021-02-08 21:42:48.267803: I tensorflow_serving/core/loader_harness.cc:87] Successfully loaded servable version {name: facial_expressions_model version: 1}
2021-02-08 21:42:48.270913: I tensorflow_serving/model_ser

### Make requests to both the models in tensorflow serving

In [ ]:
class ModelPrediction:
  def __init__(self):

  def predict(X_test):
    X_test = X_test/255.
    # Making predictions from the facialkeypoints model
    pred_keypoints = facialkeypoints_model.predict(X_test)

    # Making predictions from the expression model
    pred_expressions = np.argmax(facialexpression_model.predict(X_test), axis=-1)

    # Reshaping the array from (856, ) to (856, 1)
    pred_expressions = np.expand_dims(df_expression, axis=1)

    # Converting the predictions into a dataframe
    df_predict = pd.DataFrame(pred_keypoints, columns=columns)

    # Adding expression to the prediction dataframe
    df_predict["expression"] = pred_expressions

    return df_predict